# Importar librerias

In [1]:
import pandas as pd 

In [2]:
import time

In [3]:
from bs4 import BeautifulSoup

In [4]:
 import requests

In [5]:
from tqdm.notebook import tqdm

# Definir funciones

In [6]:
def get_data(d,m,y): # input day,month,year in String format and returns the server response 
    global soup
    url = "https://rto.cent.gov.ar/rto/RTO/generarPlanillaDiaria"
    payload = "fechaConsulta="+d+"%2F"+m+"%2F"+y+"&agruparPorLinea=NO&idTaller="
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0",
        "Accept": "text/javascript, text/html, application/xml, text/xml, */*",
        "Accept-Language": "es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "X-Requested-With": "XMLHttpRequest",
        "X-Prototype-Version": "1.6.1",
        "Content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://rto.cent.gov.ar",
        "Connection": "keep-alive",
        "Referer": "https://rto.cent.gov.ar/rto/RTO/planillaDiaria",
        "Cookie": cookie,
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin"
    }

    response = requests.request("POST", url, data=payload, headers=headers)
    soup=BeautifulSoup(response.text,'lxml')
    #time.sleep(0.5)
    return(soup)
    

In [7]:
def clean_rows():# returns a list of values: datos
        tags_datos=soup.select('td') #data is inside the tag td
        global datos
        datos=[] 
        for data in tags_datos:
            datos.append(data.get_text(strip=True)) #make a list with the values 
        return(datos)

In [8]:
def clean_columns():# returns a list of values: labels
    tags_labels=soup.select('th')
    global labels
    labels=[]
    for label in tags_labels:
        labels.append(label.get_text(strip=True)) #get list names  a list with the labels
    return(labels)   

In [9]:
def create_df():# returns a dataframe df 
    global df
    df=pd.DataFrame()
    num_col=int(14)
    num_row=int(len(datos)/num_col)
    rows=[]
    for j in range(0,num_row,1):
        row=[]
        for i in range(0,num_col,1):
            index=i+(14*j)
            row.append(datos[index])
        rows.append(row)
    #print(rows)    
    df=pd.DataFrame(rows,columns=labels)
    return(df)

In [10]:

def add_date_col(d,m,y): # adds static date column to the dataframe , input is d,m,y in STRING 
    global df
    df.insert(0,'Fecha',d+'/'+m+'/'+y)
    return(df)

def add_date_col_old(d,m,y): # adds static date column to the dataframe 
    global df
    df.insert(0,'Fecha',str(d)+'/'+str(m)+'/'+str(y))
    return(df)


In [11]:
def get_dates(s,e): # takes in a start and end string date in yyyy/mm/dd  format and returns a list of list of dates in string format 
    global dates_str
    dates=pd.date_range(start=s,end=e,freq='d')
    dates_str=[]
    date_str=[]
    for date in dates:
        year= str(date.year)
        if date.month<10:
            month='0'+str(date.month)
        else:
            month=str(date.month)
        if date.day<10:
            day='0'+str(date.day)
        else:
            day=str(date.day)
        date_str=[day,month,year]
        dates_str.append(date_str)
    return(dates_str)

In [12]:
def to_csv():
    file_name=start_date.replace('/','.')+'-'+end_date.replace('/','.')
    df_tot.to_csv(file_name)
    

In [17]:
def scrape(s,e): #  takes in a start and end string date in yyyy/mm/dd  format
    global df_tot
    global list_of_df
    df_tot=pd.DataFrame()
    list_of_df=[]
    get_dates(s,e) # returns a list of a list of dates in var dates_str
    for date in tqdm(dates_str):
        d=date[0]
        m=date[1]
        y=date[2]
        get_data(d,m,y)#returns soup
        clean_rows()#returns list of values
        create_df()#returns dataframe df 
        add_date_col(d,m,y)# adds date col to dataframe df 
        list_of_df.append(df) #adds current dataframe df to a list
    df_tot=pd.concat(list_of_df,ignore_index=True) # after all the dataframes are put into the list , unions all
   # df_tot['Fecha']=df_tot['Fecha'].astype('M')#sets Fecha column to datetime datatype
    df_tot.set_index('Planilla',inplace=True) #sets nro planilla(is unique) as index 
    return(df_tot['Fecha'].nunique()) # returns number of distinct dates with data 


# Nombre de Columnas

In [18]:
labels=['Inicio',
 'hora_Resultado',
 'Fin',
 'Tipo',
 'Uso',
 'Categoría',
 'Planilla',
 'Certificado',
 'Vence',
 'Dominio',
 'Resultado',
 'CUIT',
 'Razón Social',
 'Línea']

# Actualizar cookie

In [19]:
cookie="JSESSIONID=F21BA00B7E7B91E8D0B22F5445FB3362"

# Definir Fecha de inicio y Final

In [64]:
FORMATO='YYYY/M/D'
start_date='2022/1/1'
end_date='2022/12/31'

# Obtener Datos

In [68]:
scrape(start_date,end_date)

  0%|          | 0/365 [00:00<?, ?it/s]

294

# Verificar datos obtenidos

In [ ]:
pd.set_option('display.max_rows',None)
df_tot.head(10)

In [71]:
df_tot[df_tot['Resultado']=='Apto']['Dominio'].nunique()

18853

# Escribir datos a csv 

In [72]:
to_csv()